In [16]:
def count_apperance(result_list,data):
    c = []
    for i in result_list:
        c.append((i,0))

    for i in range(len(c)):
        for j in range(len(data)):
            if c[i][0].issubset(data[j][1]):
                c[i] = (c[i][0],c[i][1]+1)
    return c

In [18]:
def self_join(k, c):
    l = [item for item in c if item[1] >= 2]


    new = [i[0] for i in l]
    result_list = []
    # new = [{0,2,3},{0,1,2},{0,1,4},{1,2,4},{0,2,4}, {0,1,5}]
    for i in range(len(new)):
        for j in range(i+1, len(new)):
                    temp1 = list(new[i])
                    temp2 = list(new[j])
                    if(temp1[0:k-1] == temp2[0:k-1]):
                        joined = set(temp1 + temp2)
                        result_list.append(joined)
                        #second step of pruning is still missing see slide 39
    return result_list

In [1]:
from itertools import combinations, product
import csv

database = []
with open('myDataFile.csv', 'r') as file:
    reader = csv.reader(file)
    for row in reader:
        database.append(row)

data = []
for i in range(len(database)):
    data.append((i,set()))

# makes the database to workable format
for i in range(len(database)):
    for j in range(len(database[i])):
        if database[i][j] == 't':
           data[i][1].add(j)   
k = 1
c = []
for i in range(len(database[0])):
    c.append(({i},0))

for i in range(len(c)):
    for j in range(len(data)):
        if c[i][0].issubset(data[j][1]):
            c[i] = (c[i][0],c[i][1]+1)

########################
result_list = self_join(1,c)

c = count_apperance(result_list, data)

#################################
result_list = self_join(2,c)

c = count_apperance(result_list, data)
print(c)

IOPub data rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_data_rate_limit`.

Current values:
ServerApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
ServerApp.rate_limit_window=3.0 (secs)



IndexError: list index out of range